In [100]:
import cv2
import face_recognition

import csv
import os

import numpy as np
from datetime import datetime

In [102]:
target_size = (128,128)

def preprocess_image(img):
    img = cv2.resize(img, target_size)
    img = img.astype('float64') / 255
    img = np.expand_dims(img, axis=0)
    return img

In [104]:
folder = 'pictures'

known_face_names = []
known_face_encodings = []

for filename in os.listdir(folder):
    img = os.path.join(folder, filename)
    image = face_recognition.load_image_file(img)
    face_encodings = face_recognition.face_encodings(image)

    if face_encodings:
        name = os.path.splitext(filename)[0]
        known_face_encodings.append(face_encodings[0])
        known_face_names.append(name)

print(f"Load {len(known_face_encodings)} encodings")
print("Known Face Names", known_face_names)

Load 5 encodings
Known Face Names ['Anas', 'Shehryar', 'Usaid', 'Waqas', 'Zohaib']


In [105]:
attendance_file = 'attendance.csv'

def intialize_csv(attendance_file):
    if not os.path.exists(attendance_file):
        with open(attendance_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Name', 'Status', 'Time', 'Date'])

    

attendance = set()

def mark_attendance(name, status):
    now = datetime.now()
    time = now.strftime("%I:%M %p")
    date = now.strftime("%d-%m-%Y")

    if name not in attendance:
        attendance.add(name)
        with open(attendance_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([name, status, time, date])

In [106]:
def video():
    cap = cv2.VideoCapture(0) 

    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Find all face locations and encodings in the frame
        face_locations = face_recognition.face_locations(rgb)
        face_encodings = face_recognition.face_encodings(rgb, face_locations)

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # Check if the face matches any known encodings
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            distance = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(distance)  
            name = 'Unknown'
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
            
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 1)
            cv2.putText(frame, name, (left, top - 10), cv2.FONT_ITALIC, 0.9, (255, 255, 255), 1)

            
            if name != 'Unknown':
                mark_attendance(name, 'Present')

        # Display the resulting frame
        cv2.imshow('Attendance System', frame)  

        # Press 'q' to exit the loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Release the video capture and close display windows
    cap.release()
    cv2.destroyAllWindows()

def run_attendance_system():
    intialize_csv(attendance_file)
    video()

run_attendance_system()